In [9]:
import torch
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

from src.Parametric_TSNE import train_ptsne, plot_results, Network
from src.GaussianMixture import gaussian_mixture

In [2]:
PATH_SAVE_LD = 'src/Results_Uniform/Low Dimension'
PATH_SAVE_D = 'src/Results_Uniform/Density'
PATH_RESULTS = 'src/Results_Uniform'
PATH_DATA = '/home/nicolasdc/Documents/Datasets/Photon-Number-Classification/NIST CSV'

data_train = pl.read_csv(f'{PATH_DATA}/uniform_train.csv', has_header=False, separator=",").to_numpy()
data_test = pl.read_csv(f'{PATH_DATA}/uniform_test.csv', has_header=False, separator=",").to_numpy()
expected_prob = pl.read_csv(f'{PATH_DATA}/uniform_expected_prob.csv', has_header=False, separator=",").to_numpy()
db_train = pl.read_csv(f'{PATH_DATA}/db_train.csv', has_header=False, separator=",").to_numpy()
db_test = pl.read_csv(f'{PATH_DATA}/db_test.csv', has_header=False, separator=",").to_numpy()

In [ ]:
params = {
'network' : {
    'dim_input' : 350,
    'dim_emb' : 1
    },
'tSNE' : {
    'perplexity' : 10,
    'early_exaggeration' : 4,
    'early_exaggeration_constant' : 12,
    'dist_func_name' : 'euc',
    'bin_search_tol' : 1e-5,
    'bin_search_max_iter' : 300,
    'min_allowed_sig_sq' : 0,
    'max_allowed_sig_sq' : 1000,
    },
'training' : {
    'n_epochs' : 1000,
    'learning_rate' : 0.001,
    'alpha_pos' : 0,
    'alpha_l1' : 0,
    'alpha_KL' : 1,
    'size_gauss' : 4,
    'batch_size' : 3_000,
    'n_cluster' : 30
    }
}

model = Network(**params['network'])

train_ptsne(
            X_high = data_train[:30_000],
            # Model
            model = model,
            # Params
            params = params,
            # Training
            **params['training'],
            # Files
            model_name = f'test 6',
            dev = 'cpu',
            save_dir_path = 'src/Models/Sweep 1D thermal',
            # t-SNE
            **params['tSNE'],
            # Display load
            verbose = True
            )

In [ ]:
plot_results(model,
            folder_name = 'src/Models/Sweep 1D thermal',
            model_name = 'test 6')

In [ ]:
folder_name = 'src/Models/Sweep 1D thermal'
model_name = 'test 6'#file_
n_cluster = 30
X_ = np.linspace(0,1,1000)


print(model_name)

train_dl = torch.from_numpy(data_test).view(-1, 350).float()
train_dl = (train_dl - train_dl.min()) / (train_dl.max() - train_dl.min())
folder_name = f'{folder_name}/{model_name}'
model.load_state_dict(torch.load(f'{folder_name}/model.pt', weights_only=True))
model = model.to(torch.device('cpu'))
model.eval()

emb, reconst = model(train_dl, both=True)
emb_ = (emb - emb.min()) / (emb.max() - emb.min())

means = torch.linspace(0,1,n_cluster)
std = 1 / (2*n_cluster)
emb_net = emb_[:,0].view(-1,1)
desired_dist = 1.5 - torch.sum(torch.exp(- 0.5 * ((emb_net - means) / std) ** 2), dim=1)

with plt.style.context("seaborn-v0_8"):

    plt.figure(figsize=(8,4))
    emb_ = emb_.detach().numpy()
    plt.hist(emb_, alpha=0.8, bins=2000)
    plt.scatter(emb_[:,0], desired_dist.detach().numpy(), s=1)
    plt.yscale('log')
    plt.show()


test 6
